In [1]:
%env RELEASE_VERSION=0.1.18
!pip install https://storage.googleapis.com/ml-pipeline/release/${RELEASE_VERSION}/kfp.tar.gz --upgrade --user

W0407 15:40:01.764074 140064076842816 deprecation_wrapper.py:119] From /home/dkube/.ipython/extensions/myextension.py:9: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0407 15:40:01.777127 140064076842816 deprecation_wrapper.py:119] From /home/dkube/.ipython/extensions/myextension.py:10: The name tf.gfile.DeleteRecursively is deprecated. Please use tf.io.gfile.rmtree instead.



env: RELEASE_VERSION=0.1.18
     |████████████████████████████████| 153kB 5.7MB/s eta 0:00:01
     |████████████████████████████████| 122kB 8.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 12.3MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 19.2MB/s eta 0:00:01
     |████████████████████████████████| 2.7MB 30.1MB/s eta 0:00:01
     |████████████████████████████████| 92kB 18.8MB/s eta 0:00:01
     |████████████████████████████████| 61kB 16.1MB/s eta 0:00:01
     |████████████████████████████████| 204kB 31.2MB/s eta 0:00:01
     |████████████████████████████████| 163kB 30.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 16.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 39.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.7MB/s eta 0:00:01
  Stored in directory: /tmp/pip-ephem-wheel-cache-w9uxwe8t/wheels/a8/be/46/964c241809fd50c2a15d63d0d61e4c48596c075dce5a83265c
  Stored in directory: /home/dkube/.cache/pip/whe

In [1]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client

W0407 15:40:21.996864 139996295509824 deprecation_wrapper.py:119] From /home/dkube/.ipython/extensions/myextension.py:9: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.



Create a Dkube pipeline experiment

In [2]:
client = kfp.Client()
parabricks_experiment = client.create_experiment(name='human-par-pipeline')

In [16]:
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client

import os
import json
from random import randint

dkube_training_op           = components.load_component_from_file("training-component.yaml")

description = """Given one or more pairs of fastq files, you can run the human_par
pipeline workflow to generate output including BAM, recal, and variants called with
proper pseudoautosomal region ploidy values.
"""
@dsl.pipeline(
    name='human-par-pipeline',
    description=description
)
def human_par_pipeline(
    auth_token  = os.getenv("DKUBE_USER_ACCESS_TOKEN"),
    container = json.dumps({'image':'docker.io/ocdr/parabricks:v2.5.0', 'username':'', 'password': ''}),
    workspace = "parabricks",
    datasets = json.dumps(["parabricks-sample"]),
    ref = "Ref/Homo_sapiens_assembly38.fasta",
    in_fq_1 = "Data/sample_1.fq.gz",
    in_fq_2 = "Data/sample_2.fq.gz",
    knownSites = "Ref/Homo_sapiens_assembly38.known_indels.vcf.gz",
    range_male = "1-10",
    range_female = "150-250",
    sample_sex = "male",    
    #Request gpus as needed. Val 0 means no gpu,
    num_gpus=2):

    #Script to run inside the training container
    args = """--ref $DKUBE_INPUT_DATASETS/{} --in-fq $DKUBE_INPUT_DATASETS/{} $DKUBE_INPUT_DATASETS/{} \
            --knownSites $DKUBE_INPUT_DATASETS/{} --range-male {} --range-female {} --sample-sex {} \
            --out-bam $DKUBE_INPUT_DATASETS/output.bam --out-variants $DKUBE_INPUT_DATASETS/output.vcf \
            --out-recal-file $DKUBE_INPUT_DATASETS/report.txt --num-gpus {}""".format(
        ref, in_fq_1, in_fq_2,knownSites, range_male, range_female, sample_sex, num_gpus
    )
    
    #Path to NVIDIA parabricks license. Upload it to workspace"
    license_file = "license.bin"
    run_script="/INSTALL/pbrun human_par " + args + " --license-file " + license_file
    
    dkube_training_op(auth_token, container, program=workspace, run_script=run_script, datasets=datasets, ngpus=num_gpus)

Compile and generate tar ball

In [17]:
compiler.Compiler().compile(human_par_pipeline, 'human_par_pl.tar.gz')

Create and Run pipeline

In [18]:
run = client.run_pipeline(parabricks_experiment.id, 'human-par-pipeline', 'human_par_pl.tar.gz', params={})